# Install Nuget's

In [1]:
#r "nuget: Microsoft.ML, 2.0.0"

Installed Packages Microsoft.ML, 2.0.0

## Declare usings

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;

## Create Models

In [3]:
public class SalaryData
{
    [LoadColumn(0)]
    public float Experience { get; set; }

    [LoadColumn(1)]
    public string EducationLevel { get; set; }

    [LoadColumn(2)]
    public string JobTitle { get; set; }

    [LoadColumn(3)]
    public float Salary { get; set; }
}

public class SalaryPrediction
{
    [ColumnName("Score")]
    public float Salary { get; set; }
}

## Declare ML Context

In [4]:
var mlContext = new MLContext();

## Traning Data

In [5]:
var data = mlContext.Data.LoadFromTextFile<SalaryData>(path: "salary_data.csv", hasHeader: true, separatorChar: ',');

## Create and train prediction engine

In [6]:
// Define pipeline
var pipeline = mlContext.Transforms.Categorical.OneHotEncoding(new[]
    {
        new InputOutputColumnPair("EducationLevel"),
        new InputOutputColumnPair("JobTitle")
    })
    .Append(mlContext.Transforms.Concatenate("Features", "Experience", "EducationLevel", "JobTitle"))
    .Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "Salary", maximumNumberOfIterations: 100));

// Train
var model = pipeline.Fit(data);

// Prediction
var predictionEngine = mlContext.Model.CreatePredictionEngine<SalaryData, SalaryPrediction>(model);

## Test the prediction engine

In [8]:
var testData = new List<SalaryData>
{
    new SalaryData { Experience = 2, EducationLevel = "Bachelor", JobTitle = "Software Engineer" },
    new SalaryData { Experience = 4, EducationLevel = "Master", JobTitle = "Data Scientist" },
    new SalaryData { Experience = 6, EducationLevel = "PhD", JobTitle = "Research Scientist" }
};

foreach (var sample in testData)
{
    var prediction = predictionEngine.Predict(sample);
    Console.WriteLine($"Predicted Salary for {sample.JobTitle} With {sample.Experience} years of Experience: {prediction.Salary}$");
}

Predicted Salary for Software Engineer With 2 years of Experience: 62193.57$
Predicted Salary for Data Scientist With 4 years of Experience: 89980.55$
Predicted Salary for Research Scientist With 6 years of Experience: 114855.91$
